## title.basics.tsv.gz

In [ ]:
# Variablen definieren um übergebene Werte von Pipeline-Parametern aufzunehmen
run_year = ""
run_month = ""
run_day = ""

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import col

#path = "Files/2024/01/16/"
path = "Files/" + run_year + "/" + run_month + "/" + run_day + "/"

In [ ]:
filepath = path + "title.basics.tsv.gz"
title_basics = spark.read \
    .options(    
        format="csv",
        header=True,
        delimiter="\t",
        compression="gzip",
        nullValue="\\N"
    ) \
    .csv(filepath)

display(title_basics.count())
title_basics.printSchema()
display(title_basics.limit(10))

In [ ]:
# Um Reaload der Text zu umgehen sowie zwecks einfacherer Wiederverwendung von Code,
# wird Datenset einer neuen df-Variabel übergeben
df = title_basics

In [ ]:
# dient insbesondere der Überprüfung, ob Textwerte in numerischen Spalten vorkommen (Hinweis auf verrutschte Spalten)
df.summary("count", "mean", "min", "max").show()

In [ ]:
# Zeilen mit verrutschten Spalten löschen
df = df.where((df.isAdult == '0') | (df.isAdult == '1'))
display(df.count())

In [ ]:
df = df.select(
    "tconst",
    "titleType",
    "primaryTitle",
    "originalTitle",
    "isAdult",
    col("startYear").cast("short"),
    col("endYear").cast("short"),
    col("runtimeMinutes").cast("integer"),
    col("isAdult").cast("boolean").alias("isAdult1"))
df.printSchema()

In [ ]:
# sanity check - erwartet: nur Werte von 1 oder 0: 1 -> true, 0 -> false
display(df.select('isAdult', 'isAdult1').distinct().collect())

In [ ]:
df = df.drop("isAdult")
df = df.withColumnRenamed("isAdult1", "isAdult")
df.printSchema()

In [ ]:
df.write.mode("overwrite").parquet(path + "title.basics.parquet")
df.write.mode("overwrite").format("delta").saveAsTable("silver_title_basics")